In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_columns', None)

from sklearn.model_selection import train_test_split
##need to encode labels for xgboost
from sklearn.preprocessing import LabelEncoder

import xgboost as xgb

In [3]:
## Test 3: 0.79268
## Test 4: same results
## test 5: 0.76829 -- using categorical variables
## Test 7: 0.80487 After making Hospital Categorical.  Will need to lookinto removing a column or two.
## Test 8 and 9:  Removing all lesions was a step backwards.  I will try just removing one or two.
## Test 10/11 produced close to same results as 8 and 9


In [4]:
test_file = 'Input_Data/test.csv'
train_file = 'Input_Data/train.csv'

In [5]:
test_data = pd.read_csv(test_file)
train_data = pd.read_csv(train_file)

In [6]:
train_describe = train_data.copy()
test_describe = test_data.copy()

In [7]:
train_data.describe().drop(['id','hospital_number'],axis=1)

,rectal_temp,pulse,respiratory_rate,nasogastric_reflux_ph,packed_cell_volume,total_protein,abdomo_protein,lesion_1,lesion_2,lesion_3
count,1235.000000,1235.000000,1235.000000,1235.000000,1235.000000,1235.000000,1235.000000,1235.000000,1235.000000,1235.000000
mean,38.202186,79.574089,30.054251,4.382591,49.602429,21.388016,3.290931,3832.496356,14.612146,3.577328
std,0.788668,29.108638,16.452066,1.937357,10.535800,26.676453,1.589195,5436.733774,193.705735,88.858953
min,35.400000,30.000000,8.000000,1.000000,23.000000,3.500000,0.100000,0.000000,0.000000,0.000000
25%,37.800000,53.000000,18.000000,2.000000,43.000000,6.600000,2.000000,2205.000000,0.000000,0.000000
50%,38.200000,76.000000,28.000000,4.500000,48.000000,7.500000,3.000000,2209.000000,0.000000,0.000000
75%,38.600000,100.000000,36.000000,6.000000,57.000000,9.100000,4.300000,3205.000000,0.000000,0.000000
max,40.800000,184.000000,96.000000,7.500000,75.000000,89.000000,10.100000,41110.000000,3112.000000,2209.000000


In [8]:
test_describe.describe().drop(['id','hospital_number'],axis=1)

,rectal_temp,pulse,respiratory_rate,nasogastric_reflux_ph,packed_cell_volume,total_protein,abdomo_protein,lesion_1,lesion_2,lesion_3
count,824.000000,824.000000,824.00000,824.000000,824.000000,824.000000,824.000000,824.000000,824.000000,824.0
mean,38.244539,80.229369,30.71966,4.508495,49.063350,20.796238,3.336420,3709.803398,12.391990,0.0
std,0.785234,29.164711,17.43191,1.883464,10.450138,26.413588,1.539235,5112.931057,197.067771,0.0
min,36.000000,36.000000,9.00000,1.000000,23.000000,3.900000,0.100000,0.000000,0.000000,0.0
25%,37.800000,54.000000,18.00000,3.000000,43.000000,6.600000,2.000000,2205.000000,0.000000,0.0
50%,38.200000,76.000000,28.00000,4.500000,48.000000,7.500000,3.300000,2209.000000,0.000000,0.0
75%,38.600000,100.000000,36.00000,6.500000,55.000000,8.900000,4.300000,3205.000000,0.000000,0.0
max,40.800000,184.000000,96.00000,7.500000,75.000000,89.000000,10.100000,31110.000000,4300.000000,0.0


### This cell was created after the 7th test.  Wanted to know the correlation between columns

In [9]:
dummies_train = train_data.select_dtypes(include='object')
dummies_test = test_data.select_dtypes(include='object')

d_train = pd.get_dummies(train_data, columns=dummies_train.columns)
d_test = pd.get_dummies(test_data, columns=dummies_test.columns)

##not sure if hospital_number will through off the correlation.
##I removed the columns that were missing in or the other dataframe.  See cells before calculation
##Next I removed lesions because they are closest to white and there are two correlation columns for lesion_surgery.  I will rerun the calculations without lesions
##Test 8/9 are trash.  Will start trying with 3.
d_train = d_train.drop(['id','hospital_number','nasogastric_reflux_slight', 'rectal_exam_feces_serosanguious', 
                        'peristalsis_distend_small', 'pain_slight','lesion_3'],axis=1)
d_test = d_test.drop(['id','hospital_number'],axis=1)

In [10]:
d_train.corr(method='pearson').style.background_gradient(cmap='Blues')

In [11]:
##Adding missing data
##EA was done as part of another notebook.  Will try and keep the rest of the notebooks clean.
for column in train_data.select_dtypes(include=["object"]):
    train_data[column]=train_data[column].fillna("no_measure_recorded")
    
for column in test_data.select_dtypes(include=["object"]):
    test_data[column]=test_data[column].fillna("no_measure_recorded")
    
for column in train_data.select_dtypes(include=["int64"]):
    train_data[column]=train_data[column].fillna(0)
    
for column in test_data.select_dtypes(include=["int64"]):
    test_data[column]=test_data[column].fillna(0)

for column in train_data.select_dtypes(include=["float64"]):
    train_data[column]=train_data[column].fillna(0)
    
for column in test_data.select_dtypes(include=["float64"]):
    test_data[column]=test_data[column].fillna(0)

In [12]:
def remove_missing_columns(df,column_set):
    for column in column_set:
        if column in df.columns:
            df = df.drop(columns=column)
    return df        
    

missing_columns={'nasogastric_reflux_slight', 'rectal_exam_feces_serosanguious', 'peristalsis_distend_small', 'pain_slight','pain_moderate',"lesion_3"}

test_data = remove_missing_columns(test_data,missing_columns)
train_data = remove_missing_columns(train_data,missing_columns)

# test_data = test_data.drop(columns=missing_columns)
# train_data = train_data.drop(columns=missing_columns)


In [13]:
le = LabelEncoder()
train_data['encoded_outcome'] = le.fit_transform(train_data['outcome'])
y = train_data['encoded_outcome']
train_data.head()
test_data.head()
y.shape

(1235,)

In [14]:
trained_dropped = train_data.drop(['id','encoded_outcome','outcome'],axis=1) ## Tried with and without hospital.
test_dropped = test_data.drop(['id'], axis=1)


trainedc = trained_dropped.copy()
testc = test_dropped.copy()
trained_dropped.shape,test_dropped.shape

((1235, 26), (824, 26))

In [15]:
## Didn't realize I needed to turn things into dummies
train_dummy_objects = trained_dropped.select_dtypes(include='object')
test_dummy_objects = test_dropped.select_dtypes(include='object')
print(train_dummy_objects.shape,test_dummy_objects.shape
)
test_with_dummies = pd.get_dummies(test_dropped, columns=test_dummy_objects.columns)
train_with_dummies = pd.get_dummies(trained_dropped, columns=train_dummy_objects.columns)
print(test_with_dummies.shape,train_with_dummies.shape)

#category
trained_cats = trainedc.select_dtypes(include=['object']).columns
test_cats = testc.select_dtypes(include=['object']).columns

trainedc[trained_cats] = trainedc[trained_cats].astype('category')
testc[test_cats] = testc[test_cats].astype('category')

trainedc['hospital_number'] = trainedc['hospital_number'].astype('category')
testc['hospital_number'] = testc['hospital_number'].astype('category')



print(trainedc.shape,testc.shape)

(1235, 16) (824, 16)
(824, 79) (1235, 82)
(1235, 26) (824, 26)


In [16]:
def remove_missing_columns(df,column_set):
    for column in column_set:
        if column in df.columns:
            df = df.drop(columns=column)
    return df        
    

missing_columns={'nasogastric_reflux_slight', 'rectal_exam_feces_serosanguious', 'peristalsis_distend_small', 
                 'pain_slight','pain_moderate','lesion_3'}#'lesion_1','lesion_2','lesion_3'

test_with_dummies = remove_missing_columns(test_with_dummies,missing_columns)
train_with_dummies = remove_missing_columns(train_with_dummies,missing_columns)

testc = remove_missing_columns(testc,missing_columns)
trainedc = remove_missing_columns(trainedc,missing_columns)



In [17]:
X = train_with_dummies
X.shape
Xcat = trainedc
Xcat.shape

(1235, 26)

In [18]:
X.dtypes

hospital_number                      int64
rectal_temp                        float64
pulse                              float64
respiratory_rate                   float64
nasogastric_reflux_ph              float64
                                    ...   
abdomo_appearance_serosanguious      uint8
surgical_lesion_no                   uint8
surgical_lesion_yes                  uint8
cp_data_no                           uint8
cp_data_yes                          uint8
Length: 78, dtype: object

In [19]:
xgb_model = xgb.XGBClassifier(learning_rate=0.1, n_estimator=1000, max_depth=3,
                              objective='binary:logistic',random_state=42)
xgb_model.fit(X,y)
xgb_model_cat = xgb.XGBClassifier(tree_method="hist", enable_categorical=True,device="cuda")
xgb_model_cat.fit(Xcat,y)

C:\Users\awnet\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [10:10:02] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimator" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cuda', early_stopping_rounds=None,
              enable_categorical=True, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [20]:
test_with_dummies.shape

(824, 78)

In [21]:
train_with_dummies.shape

(1235, 78)

In [22]:
y_xgb_pred = xgb_model.predict(test_with_dummies)
y_xgb_pred_cat = xgb_model_cat.predict(testc)

C:\Users\awnet\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [10:10:09] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


In [23]:
# y_xgb_pred_cat
# y_xgb_pred

In [24]:
labels = ['died', 'euthanized', 'lived']

leII = LabelEncoder()
leII.fit(labels)

encoded_label = leII.transform(labels)

ypred_labels = leII.inverse_transform(y_xgb_pred)
ypred_cat_labels = leII.inverse_transform(y_xgb_pred_cat)


In [25]:
ypcat_labels = test_data.copy()

ypcat_labels['outcome'] = ypred_cat_labels
test_data['outcome'] = ypred_labels

In [26]:
# output_y = test_data[['id','outcome']]
# output_y.to_csv('outputXII.csv',index=False)

# output_ycat = ypcat_labels[['id','outcome']]
# output_ycat.to_csv('outputXIII.csv',index=False)
